In [161]:
### Execute TRANFAC enrichment analysis based on co-eqtl results

# Libraries

In [162]:
source('MS1_Libraries.r')

# Parameters

In [163]:
### Path to input data

In [164]:
path<-""
outdir<-""

# Data 

## Enrichment Data Input

In [165]:
### Exemplary data input load for a cell-type

In [166]:
cell_type_var = "CD4T"
# c("CD4T","CD8T","monocyte","NK","B","DC")

In [167]:
for(cell_type in cell_type_var){

  coeqtls <- fread(paste0(path, "UT_",cell_type, 
                         "_coeqtls_fullresults_fixed.all.tsv.gz"))
  coeqtls$gene1<-gsub(";.*","",coeqtls$Gene)
  coeqtls$gene2<-gsub(".*;","",coeqtls$Gene)
  coeqtls$second_gene<-ifelse(coeqtls$gene1 == coeqtls$eqtlgen, coeqtls$gene2,
                        coeqtls$gene1)
  coeqtls$gene1<-NULL
  coeqtls$gene2<-NULL
    }

In [168]:
#unique(coeqtls$eqtlgene)

In [169]:
nrow(coeqtls[(coeqtls$eqtlgene ==  'RPS26') &  (coeqtls$gene2_isSig == TRUE),c('eqtlgene', 'second_gene')])

# validity check --> 372 significant co-egenes for RPS26

[1] 372

In [170]:
nrow(coeqtls[(coeqtls$eqtlgene ==  'RPS26'),c('eqtlgene', 'second_gene')])
# overall 742 --> those that would not haven been tested

[1] 742

In [171]:
head(coeqtls,2)

snp_genepair,Gene,GeneChr,GenePos,GeneStrand,GeneSymbol,SNP,SNPChr,SNPPos,SNPAlleles,⋯,multipletestP,eqtlgene,snp_eqtlgene,snp_beta_shape1,snp_beta_shape2,snp_pvalbeta,snp_qval,gene2_pthreshold,gene2_isSig,second_gene
<chr>,<chr>,<int>,<int>,<lgl>,<chr>,<chr>,<int>,<int>,<chr>,⋯,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>
rs11587831_C1orf86;NUDT22,C1orf86;NUDT22,1,2115903,NA,C1orf86;NUDT22,rs11587831,1,2110848,T/G,⋯,0.6354470,C1orf86,rs11587831_C1orf86,1.197903,127.555,0.5044989,0.7012273,4.539067e-05,FALSE,NUDT22
rs11587831_C1orf86;SDHC,C1orf86;SDHC,1,2115903,NA,C1orf86;SDHC,rs11587831,1,2110848,T/G,⋯,0.9144163,C1orf86,rs11587831_C1orf86,1.197903,127.555,0.5044989,0.7012273,4.539067e-05,FALSE,SDHC


## ReMap Results for comparison

In [173]:
## Load supplementary table (with ReMap Results to compare):
# "supptable15.TFenrichment_co-eGenes.xlsx - Sheet1.csv"

In [174]:
old_enrichments = read.csv( paste0(path, "supptable15.TFenrichment_co-eGenes.xlsx - Sheet1.csv"))

In [175]:
nrow(old_enrichments)

[1] 963

In [176]:
head(old_enrichments,2)

,Cell.type,eQTL..SNP.eGene.,TF,TF.is.a.co.eGene.,enrichment.p.value,X..TF.overlap...co.eGene,X..TF.overlap...background,X..no.TF.overlap...co.eGene,X..background.gene...not.co.eGene,enrichment.fdr,eQTL.SNP,SNP.overlaps.TF.,Names.of.overlapping.SNPs
,<chr>,<chr>,<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<lgl>,<chr>
1,CD4T,rs111454690_HLA-DRB5,CDK8,FALSE,9.630369e-06,14,5,2778,8515,1.640373e-03,rs111454690,FALSE,
2,CD4T,rs111454690_HLA-DRB5,SNRNP70,FALSE,1.209254e-09,11,8,649,10644,6.179288e-07,rs111454690,FALSE,


In [177]:
## Check out some results of ReMap mentioned in paper

In [178]:
max(old_enrichments$enrichment.p.value)

[1] 0.01323472

In [179]:
max(old_enrichments$enrichment.fdr)
# check to use same cut-off for TRANSFAC  --> 0.05

[1] 0.04983139

In [180]:
unique(old_enrichments[,c( 'eQTL..SNP.eGene.')])

[1] "rs111454690_HLA-DRB5"     "rs1131017_RPS26"         
[3] "rs4147638_SMDT1"          "rs7605824_SH3YL1"        
[5] "rs7632486_CMTM8"          "rs9271520_HLA-DQA2"      
[7] "rs1131017_RPS26_positive" "rs1131017_RPS26_negative"
[9] "rs6708265_PASK"

In [181]:
length(unique(old_enrichments[,c( 'eQTL..SNP.eGene.')]))  # subtract positive and negative case for RPS26 --> yields the 7 mentioned in paper for which there were significant TF enrichments

[1] 9

In [182]:
unique(old_enrichments[old_enrichments$SNP.overlaps.TF. == TRUE,c('eQTL..SNP.eGene.')])  # results in the 4 pairs mentioned in paper

[1] "rs1131017_RPS26"          "rs4147638_SMDT1"         
[3] "rs7605824_SH3YL1"         "rs9271520_HLA-DQA2"      
[5] "rs1131017_RPS26_positive" "rs1131017_RPS26_negative"

In [183]:
## rs1131017–RPS26 examples: RMB39, TCF7, LEF1, KLF6, CD74, MAF

In [184]:
old_enrichments[(old_enrichments$eQTL..SNP.eGene. %in% c('rs1131017_RPS26')) & (old_enrichments$TF.is.a.co.eGene. == TRUE)  & ((old_enrichments$SNP.overlaps.TF. == TRUE)),]

,Cell.type,eQTL..SNP.eGene.,TF,TF.is.a.co.eGene.,enrichment.p.value,X..TF.overlap...co.eGene,X..TF.overlap...background,X..no.TF.overlap...co.eGene,X..background.gene...not.co.eGene,enrichment.fdr,eQTL.SNP,SNP.overlaps.TF.,Names.of.overlapping.SNPs
,<chr>,<chr>,<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<lgl>,<chr>
19,CD4T,rs1131017_RPS26,MAF,TRUE,3.654557e-06,92,280,1747,9546,5.187441e-05,rs1131017,TRUE,rs1131017
34,CD4T,rs1131017_RPS26,RBM39,TRUE,2.128100e-06,244,128,6041,5252,3.295330e-05,rs1131017,TRUE,"rs10876864,rs1131017,rs7297175"
50,CD4T,rs1131017_RPS26,TCF7,TRUE,7.468026e-03,134,238,3379,7914,3.052929e-02,rs1131017,TRUE,rs1131017
84,CD4T,rs1131017_RPS26,LEF1,TRUE,4.859147e-05,153,219,3529,7764,4.598193e-04,rs1131017,TRUE,"rs10876864,rs1131017"
116,CD4T,rs1131017_RPS26,KLF6,TRUE,1.597304e-03,139,233,3385,7908,8.236538e-03,rs1131017,TRUE,"rs10876864,rs1131017,rs7297175"
119,CD4T,rs1131017_RPS26,CD74,TRUE,3.954534e-06,172,200,3915,7378,5.461532e-05,rs1131017,TRUE,rs1131017
730,monocyte,rs1131017_RPS26,CD74,TRUE,7.422301e-03,63,69,3526,6028,3.134542e-02,rs1131017,TRUE,rs1131017


In [185]:
# MAF and CD74 only negative effect directions

In [186]:
# TMEM176A nothing found with remap

# Run TRANSFAC enrichment for all cell-types

In [213]:
### Set parameters for function

In [214]:
p_val_thres = 0.05

In [215]:
correction_var = 'fdr'

In [216]:
### Decide on whether to restrict the background set
restrict_background_set = FALSE

# set to TRUE for adaption

In [217]:
### Run enrichments

In [218]:
enrichment<-NULL
enrichment_summary<-NULL
coegenes_counts_total<-NULL
for(cell_type in c("DC","CD4T","CD8T","monocyte","NK","B" )){
  # Read in the data
  coeqtls <- fread(paste0(path, "UT_",cell_type, 
                         "_coeqtls_fullresults_fixed.all.tsv.gz"))
  coeqtls$gene1<-gsub(";.*","",coeqtls$Gene)
  coeqtls$gene2<-gsub(".*;","",coeqtls$Gene)
  coeqtls$second_gene<-ifelse(coeqtls$gene1 == coeqtls$eqtlgen, coeqtls$gene2,
                        coeqtls$gene1)
  coeqtls$gene1<-NULL
  coeqtls$gene2<-NULL
  
  # Take all tested genes as background
  background_genes  <- union(coeqtls$eqtlgen,coeqtls$second_gene)
  
  coeqtls_sign<-coeqtls[coeqtls$gene2_isSig,]
  
  print(paste(cell_type,"with",nrow(coeqtls_sign),"co-eQTLs"))
  
  # Identify all eQTLs with at least 5 coeGenes
  coegene_count<-coeqtls_sign%>%
    group_by(snp_eqtlgene)%>%
    summarise(count_coeGenes=n())%>%
    filter(count_coeGenes>4)
  
  coegene_count$cell_type<-cell_type
  coegenes_counts_total<-rbind(coegenes_counts_total,
                               coegene_count)
  
  enrichment_found<-0
  #Perform GO enrichemt separately for each eQTL
  for(eqtl in coegene_count$snp_eqtlgene){
      print(eqtl)
      
    # Optional restricted background set
    if(restrict_background_set == TRUE){
        background_genes = unique(c(coeqtls$eqtlgene[coeqtls$snp_eqtlgene ==  eqtl], coeqtls$second_gene[coeqtls$snp_eqtlgene ==  eqtl]))
        }
      print(length(background_genes))
    
    # Run enrichment analysis with background set
    enrich_out <- gost(
                                coeqtls_sign$second_gene[coeqtls_sign$snp_eqtlgene == eqtl],
                                organism = "hsapiens",
                                ordered_query = FALSE,
                                multi_query = FALSE,
                                significant = TRUE,
                                exclude_iea = FALSE,
                                measure_underrepresentation = FALSE,
                                evcodes = FALSE,
                                correction_method = correction_var,
                                user_threshold = p_val_thres,
                                custom_bg = background_genes,
                                sources = 'TF'   # only do transfac enrichment
                                )
    
    #if(nrow(enrich_out$result[enrich_out$result$source == 'TF',])>0){
    if(!is.null(enrich_out)){
      # Save if a enrichment was found
      enrichment_found<-enrichment_found+1
      
      # Save result dataframe
      res<-enrich_out$result[enrich_out$result$source == 'TF',]
      res$cell_type<-cell_type
      res$snp_eGene<-eqtl
      enrichment<-rbind(enrichment,
                        res)
    }

  }
  
  enrichment_summary<-rbind(enrichment_summary,
                            data.frame(cell_type,
                                       n_eqtls_freq=nrow(coegene_count),
                                       n_enrich=enrichment_found,
                                       freq_enrich=enrichment_found/nrow(coegene_count)))
  
    
    
  #Check for CD4T specificallly for RPS26 the positive & negative coeGenes separately
  if(cell_type=="CD4T"){
    eqtl<-"rs1131017_RPS26"
    
    #Test positive coeGenes (MAF not correctly flipped here)
    enrich_out <-gost(
                        coeqtls_sign$second_gene[coeqtls_sign$snp_eqtlgene == eqtl &
                                                          coeqtls_sign$MetaPZ < 0],
                        organism = "hsapiens",
                        ordered_query = FALSE,
                        multi_query = FALSE,
                        significant = TRUE,
                        exclude_iea = FALSE,
                        measure_underrepresentation = FALSE,
                        evcodes = FALSE,
                        correction_method = correction_var,
                        user_threshold = p_val_thres,
                        custom_bg = background_genes,
                        sources = 'TF'   # only do transfac enrichment
                        )
      
      
      
    
    if(!is.null(enrich_out)){
      
      # Save if a enrichment was found
      enrichment_found<-enrichment_found+1
      
      # Save result dataframe
      res<- enrich_out$result[enrich_out$result$source == 'TF',]
      res$cell_type<-cell_type
      res$snp_eGene<-paste0(eqtl,"_positive")
      enrichment<-rbind(enrichment,
                        res)
    }
    
    #Test negative coeGenes (MAF not correctly flipped here)
    enrich_out <-gost(
                       coeqtls_sign$second_gene[coeqtls_sign$snp_eqtlgene == eqtl &
                                                          coeqtls_sign$MetaPZ > 0],
                        organism = "hsapiens",
                        ordered_query = FALSE,
                        multi_query = FALSE,
                        significant = TRUE,
                        exclude_iea = FALSE,
                        measure_underrepresentation = FALSE,
                        evcodes = FALSE,
                        correction_method = correction_var,
                        user_threshold = p_val_thres,
                        custom_bg = background_genes,
                        sources = 'TF'   # only do transfac enrichment
                        )
    
    if(!is.null(enrich_out)){
      
      # Save if a enrichment was found
      enrichment_found<-enrichment_found+1
      
      # Save result dataframe
      res<-enrich_out$result[enrich_out$result$source == 'TF',]
      res$cell_type<-cell_type
      res$snp_eGene<-paste0(eqtl,"_negative")
      enrichment<-rbind(enrichment,
                        res)
    }
  }
  
 
  }

[1] "DC with 58 co-eQTLs"
[1] "rs7935082_MS4A7"
[1] 6054


Detected custom background input, domain scope is set to 'custom'



[1] "rs9271520_HLA-DQA2"
[1] 6054


Detected custom background input, domain scope is set to 'custom'

No results to show
Please make sure that the organism is correct or set significant = FALSE



[1] "CD4T with 500 co-eQTLs"
[1] "rs111454690_HLA-DRB5"
[1] 11300


...

Detected custom background input, domain scope is set to 'custom'



[1] "rs62480001_MYOM2"
[1] 7271


Detected custom background input, domain scope is set to 'custom'

No results to show
Please make sure that the organism is correct or set significant = FALSE



[1] "B with 35 co-eQTLs"
[1] "rs1131017_RPS26"
[1] 1729


Detected custom background input, domain scope is set to 'custom'



In [219]:
### Inspect result

In [220]:
head(enrichment)

,query,significant,p_value,term_size,query_size,intersection_size,precision,recall,term_id,source,term_name,effective_domain_size,source_order,parents,cell_type,snp_eGene
,<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<list>,<chr>,<chr>
1,query_1,TRUE,0.049610826,2342,27,22,0.8148148,0.009393681,TF:M00665,TF,Factor: Sp3; motif: ASMCTTGGGSRGGG,5705,7882,TF:M00000,DC,rs7935082_MS4A7
2,query_1,TRUE,0.049610826,2303,27,22,0.8148148,0.009552757,TF:M03582,TF,Factor: TWIST; motif: CACCTGG,5705,8844,TF:M00000,DC,rs7935082_MS4A7
3,query_1,TRUE,0.003978389,3447,351,163,0.4643875,0.047287496,TF:M11438,TF,Factor: SAP-1; motif: NTCGTAAATGCN,10167,1882,TF:M00000,CD4T,rs1131017_RPS26
4,query_1,TRUE,0.022537569,3025,20,16,0.8000000,0.005289256,TF:M08413,TF,Factor: TEF-3:C/EBPdelta; motif: RGWATGYNRTTRCGYAAY,10167,8434,TF:M00000,CD4T,rs7605824_SH3YL1
5,query_1,TRUE,0.002470867,3285,191,95,0.4973822,0.028919330,TF:M10785,TF,Factor: hoxa9; motif: RTCGTWANNN,10167,3774,TF:M00000,CD4T,rs1131017_RPS26_positive
6,query_1,TRUE,0.003339438,1184,191,46,0.2408377,0.038851351,TF:M04696_1,TF,Factor: YY1; motif: GCCGCCATNTTGNNNNNGGNCN; match class: 1,10167,9013,TF:M04696,CD4T,rs1131017_RPS26_positive


In [221]:
head(enrichment_summary)

,cell_type,n_eqtls_freq,n_enrich,freq_enrich
,<chr>,<int>,<dbl>,<dbl>
1,DC,2,1,0.5000000
2,CD4T,8,2,0.2500000
3,CD8T,6,0,0.0000000
4,monocyte,9,1,0.1111111
5,NK,3,2,0.6666667
6,B,1,1,1.0000000


In [223]:
head(coegenes_counts_total)

snp_eqtlgene,count_coeGenes,cell_type
<chr>,<int>,<chr>
rs7935082_MS4A7,30,DC
rs9271520_HLA-DQA2,13,DC
rs111454690_HLA-DRB5,19,CD4T
rs1131017_RPS26,372,CD4T
rs2741159_KRT1,8,CD4T
rs4147638_SMDT1,19,CD4T


In [225]:
max(enrichment$p_value)  # set to same level

[1] 0.04961083

In [226]:
### Evaluate amount of enrichments found per cell-type with set p-value threshold

In [227]:
enrichment %>% group_by(cell_type) %>% count()

cell_type,n
<chr>,<int>
B,3
CD4T,54
DC,2
monocyte,40
NK,21


In [228]:
nrow(enrichment)

[1] 120

In [230]:
### Save the enrichment result

In [231]:
enrichment$parents = NULL

In [232]:
write.csv(enrichment, paste0(path, "transfac_results/TRANSFAC_Enrichments.csv"))

# Compare to previous enrichment results with Remap

In [233]:
head(old_enrichments,2)

,Cell.type,eQTL..SNP.eGene.,TF,TF.is.a.co.eGene.,enrichment.p.value,X..TF.overlap...co.eGene,X..TF.overlap...background,X..no.TF.overlap...co.eGene,X..background.gene...not.co.eGene,enrichment.fdr,eQTL.SNP,SNP.overlaps.TF.,Names.of.overlapping.SNPs
,<chr>,<chr>,<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<lgl>,<chr>
1,CD4T,rs111454690_HLA-DRB5,CDK8,FALSE,9.630369e-06,14,5,2778,8515,1.640373e-03,rs111454690,FALSE,
2,CD4T,rs111454690_HLA-DRB5,SNRNP70,FALSE,1.209254e-09,11,8,649,10644,6.179288e-07,rs111454690,FALSE,


## Compare amount of enrichments

In [234]:
amount_enrichments_old = old_enrichments %>% group_by(Cell.type, eQTL..SNP.eGene.) %>% count()

In [235]:
head(amount_enrichments_old)

Cell.type,eQTL..SNP.eGene.,n
<chr>,<chr>,<int>
B,rs1131017_RPS26,82
CD4T,rs111454690_HLA-DRB5,14
CD4T,rs1131017_RPS26,134
CD4T,rs1131017_RPS26_negative,93
CD4T,rs1131017_RPS26_positive,125
CD4T,rs4147638_SMDT1,14


In [236]:
colnames(amount_enrichments_old) = c('cell_type', 'snp_eGene', 'ReMap_amount')

In [237]:
transfac_enrichments = enrichment %>% group_by(cell_type, snp_eGene) %>% count()

In [238]:
colnames(transfac_enrichments)= c('cell_type', 'snp_eGene', 'TRANSFAC_amount')

In [239]:
overview = merge(amount_enrichments_old, transfac_enrichments, all.x = TRUE, all.y = TRUE)

In [240]:
### Result of comparisoon

In [245]:
overview[is.na(overview)]= 0

In [246]:
overview[order(overview$TRANSFAC_amount, decreasing = TRUE),]

,cell_type,snp_eGene,ReMap_amount,TRANSFAC_amount
,<chr>,<chr>,<dbl>,<dbl>
5,CD4T,rs1131017_RPS26_positive,125,51
16,monocyte,rs1131017_RPS26,145,40
18,NK,rs1131017_RPS26,132,20
1,B,rs1131017_RPS26,82,3
14,DC,rs7935082_MS4A7,0,2
3,CD4T,rs1131017_RPS26,134,1
4,CD4T,rs1131017_RPS26_negative,93,1
7,CD4T,rs7605824_SH3YL1,58,1
19,NK,rs12151742_GNLY,0,1


In [247]:
write.csv(overview, paste0(path, "transfac_results/TRANSFAC_ReMap_comparison.csv"))

## Compare the TFs

In [254]:
# Paper: six TFs—RBM39, TCF7, LEF1, KLF6, CD74 and MAF—whose binding sites were enriched in the promoter region of the rs1131017–RPS26


In [280]:
enrichment$tf = str_extract(enrichment$term_name, '.*;')

In [281]:
enrichment$tf = str_replace(enrichment$tf, 'Factor: ', '')

In [282]:
enrichment$tf = str_replace(enrichment$tf, ';', '')

In [283]:
enrichment$tf = str_replace(enrichment$tf , 'motif.*', '')

In [284]:
enrichment$tf = str_replace(enrichment$tf , '-', '')

In [285]:
enrichment$tf = toupper(enrichment$tf)

In [286]:
enrichment$tf = str_replace(enrichment$tf , ' ', '')

In [292]:
enrichment$tf = str_replace(enrichment$tf, 'CETS-1', 'ETS1')
enrichment$tf = str_replace(enrichment$tf, 'C/EBPBETA|C/EBPBETA|C/EBPbeta|C/EBPBETA|GCMA:CEBPB', 'CEBPB')
enrichment$tf = str_replace(enrichment$tf, 'C/EBPDELTA|C/EBPDELTA|TEF3:CEBPD', 'CEBPD')
enrichment$tf = str_replace(enrichment$tf, 'C/EBPGAMMA', 'CEBPG')
enrichment$tf = str_replace(enrichment$tf, 'ELK1:HOXB13', 'ELK1')
enrichment$tf = str_replace(enrichment$tf, 'GTF2IRD1ISOFORM2', 'GTF2I')
enrichment$tf = str_replace(enrichment$tf, 'MEIS1:ELF1', 'ELF1')
enrichment$tf = str_replace(enrichment$tf, 'PU.1', 'SPI1')
enrichment$tf = str_replace(enrichment$tf, 'TEF3:ERG', 'ERG')

In [293]:
head(enrichment,2)

,query,significant,p_value,term_size,query_size,intersection_size,precision,recall,term_id,source,term_name,effective_domain_size,source_order,cell_type,snp_eGene,tf
,<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>
1,query_1,TRUE,0.04961083,2342,27,22,0.8148148,0.009393681,TF:M00665,TF,Factor: Sp3; motif: ASMCTTGGGSRGGG,5705,7882,DC,rs7935082_MS4A7,SP3
2,query_1,TRUE,0.04961083,2303,27,22,0.8148148,0.009552757,TF:M03582,TF,Factor: TWIST; motif: CACCTGG,5705,8844,DC,rs7935082_MS4A7,TWIST


In [299]:
colnames(enrichment) = paste0('TRANSFAC_', colnames(enrichment))

In [296]:
### Merge with ReMap REsults

In [298]:
head(old_enrichments,2)

,Cell.type,eQTL..SNP.eGene.,TF,TF.is.a.co.eGene.,enrichment.p.value,X..TF.overlap...co.eGene,X..TF.overlap...background,X..no.TF.overlap...co.eGene,X..background.gene...not.co.eGene,enrichment.fdr,eQTL.SNP,SNP.overlaps.TF.,Names.of.overlapping.SNPs
,<chr>,<chr>,<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<lgl>,<chr>
1,CD4T,rs111454690_HLA-DRB5,CDK8,FALSE,9.630369e-06,14,5,2778,8515,1.640373e-03,rs111454690,FALSE,
2,CD4T,rs111454690_HLA-DRB5,SNRNP70,FALSE,1.209254e-09,11,8,649,10644,6.179288e-07,rs111454690,FALSE,


In [300]:
colnames(old_enrichments) = paste0('ReMap', colnames(old_enrichments))

In [301]:
combined = merge(enrichment, old_enrichments, by.x = c('TRANSFAC_cell_type', 'TRANSFAC_snp_eGene', 'TRANSFAC_tf'), by.y = c('ReMapCell.type', 'ReMapeQTL..SNP.eGene.', 'ReMapTF'))

In [303]:
nrow(combined)

[1] 31

In [306]:
unique(combined$TRANSFAC_tf)

[1] "CEBPD" "CEBPB" "ELK1"  "FLI1"  "HOXA9"

In [304]:
combined

TRANSFAC_cell_type,TRANSFAC_snp_eGene,TRANSFAC_tf,TRANSFAC_query,TRANSFAC_significant,TRANSFAC_p_value,TRANSFAC_term_size,TRANSFAC_query_size,TRANSFAC_intersection_size,TRANSFAC_precision,⋯,ReMapTF.is.a.co.eGene.,ReMapenrichment.p.value,ReMapX..TF.overlap...co.eGene,ReMapX..TF.overlap...background,ReMapX..no.TF.overlap...co.eGene,ReMapX..background.gene...not.co.eGene,ReMapenrichment.fdr,ReMapeQTL.SNP,ReMapSNP.overlaps.TF.,ReMapNames.of.overlapping.SNPs
<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<dbl>,⋯,<lgl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<lgl>,<chr>
B,rs1131017_RPS26,CEBPD,query_1,TRUE,0.028703406,501,35,23,0.6571429,⋯,FALSE,3.034184e-06,34,1,1096,632,1.107477e-04,rs1131017,TRUE,"rs1131017,rs7297175"
CD4T,rs1131017_RPS26_positive,CEBPB,query_1,TRUE,0.028742923,2503,191,70,0.3664921,⋯,FALSE,2.421193e-05,159,41,7460,3833,2.877279e-04,rs1131017,TRUE,rs7297175
CD4T,rs1131017_RPS26_positive,CEBPB,query_1,TRUE,0.041618610,1598,191,49,0.2565445,⋯,FALSE,2.421193e-05,159,41,7460,3833,2.877279e-04,rs1131017,TRUE,rs7297175
CD4T,rs1131017_RPS26_positive,CEBPB,query_1,TRUE,0.017941417,2826,191,78,0.4083770,⋯,FALSE,2.421193e-05,159,41,7460,3833,2.877279e-04,rs1131017,TRUE,rs7297175
CD4T,rs1131017_RPS26_positive,CEBPD,query_1,TRUE,0.041618610,1464,191,46,0.2408377,⋯,FALSE,7.264122e-05,133,67,5970,5323,7.423933e-04,rs1131017,TRUE,"rs1131017,rs7297175"
CD4T,rs1131017_RPS26_positive,ELK1,query_1,TRUE,0.007142316,4467,191,113,0.5916230,⋯,FALSE,7.289311e-04,94,106,4030,7263,5.173386e-03,rs1131017,TRUE,rs10876864
CD4T,rs1131017_RPS26_positive,FLI1,query_1,TRUE,0.024172382,2207,191,64,0.3350785,⋯,FALSE,9.006657e-03,131,69,6433,4860,4.002088e-02,rs1131017,TRUE,rs1131017
CD4T,rs1131017_RPS26_positive,FLI1,query_1,TRUE,0.047267440,3060,191,80,0.4188482,⋯,FALSE,9.006657e-03,131,69,6433,4860,4.002088e-02,rs1131017,TRUE,rs1131017
CD4T,rs1131017_RPS26_positive,FLI1,query_1,TRUE,0.041618610,2325,191,65,0.3403141,⋯,FALSE,9.006657e-03,131,69,6433,4860,4.002088e-02,rs1131017,TRUE,rs1131017


In [305]:
write.csv(combined, paste0(path, "transfac_results/TRANSFAC_ReMap_matches.csv"))